# Python Coding Exercise for Module 2, Lesson 2: Subset Selection

Create a Python function that performs forward stepwise selection to identify a subset of predictor variables that are most related to a target variable in a linear regression context.

### Your tasks are to write a function that satisfies the following:

- Accept three parameters: X, a pandas DataFrame of predictor variables; y, a pandas Series of the target variable; and criterion, a string that specifies the metric used for variable selection (e.g., 'AIC', 'BIC', 'R-squared').
- Start with an empty model and iteratively add the variable that improves the model the most based on the specified criterion until no variables improve the model.
- Return a list of models with scores.

### Constraints:

- Use the statsmodels library to fit linear regression models.
- Assume that X and y have been preprocessed and are ready for modeling (e.g., no missing values or categorical variables needing encoding).

### Important Considerations:

Although we have not discussed this yet, the smaller AIC and BIC scores are, the better. So you will need to adjust the previous code of forward stepwise selection to choose smaller values over larger ones. You are also provided with several functions that you can use to answer the questions. 

In [ ]:
import pandas as pd
from sklearn.datasets import make_regression
import statsmodels.api as sm

Below is a function you can use inside forward_stepwise_selection to score your models. 

In [ ]:
def score(criterion, model):
    if criterion == 'AIC':
        return model.aic 
    if criterion == 'BIC':
        return model.bic 
    if criterion == 'R-squared':
        return -model.rsquared # negative since higher is better

In [ ]:
def fit_model(selected_features):
        model = sm.OLS(y, sm.add_constant(X[selected_features])).fit()
        return model

def forward_stepwise_selection(data, response_variable, criterion='AIC'):
    # Initialize variables
    remaining_predictors = list(data.columns)
    current_predictors = []
    models = []

    # Loop through the features
    for i in range(data.shape[1]):
        # Initialize best_candidate_score, best_candidate, and trial_columns
        # Keep in mind that smaller scores are better. What should the initial score be?
        best_candidate_score = float('inf')
        best_candidate = None
        trial_columns = list(current_predictors)  # Create a copy to avoid side effects

        # for each predictor, consider the addition, the model, and the associated score
        for predictor in remaining_predictors:
            trial_columns.append(predictor)

            model = fit_model(trial_columns)
            criterion_value = score(criterion, model)

            # if the score is better, update best_candidate and best_candidate_score
            if criterion_value < best_candidate_score:
                best_candidate_score = criterion_value
                best_candidate = predictor
            
            trial_columns.remove(predictor)
        
        if best_candidate:
            current_predictors.append(best_candidate)
            remaining_predictors.remove(best_candidate)
            models.append((list(current_predictors), best_candidate_score))

    # return the list of models, sorted from low to high!
    return sorted(models, key=lambda x: x[1])

In [ ]:
# Generate a dataset for demonstration (with 100 samples and 10 features)
X, y = make_regression(n_samples=100, n_features=10, noise=0.1, random_state=42)
X = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(1, 11)])
y = pd.Series(y, name='target')

In [ ]:
# What do the statistics on the full model tell us?

full_model = fit_model([f'feature_{i}' for i in range(1, 11)])
print(full_model.summary())

In [ ]:
# Function call
models = forward_stepwise_selection(X, y, criterion='R-squared')
print(models)

Here, we can plot the list models by considering their number of features and their associated score. What does this graph suggest?

In [ ]:
import matplotlib.pyplot as plt

def plot_features_vs_score(data):
    # Unpack the number of features and their associated scores
    num_features = [len(features) for features, score in data]
    scores = [score for features, score in data]
    
    # Create a plot
    plt.figure(figsize=(10, 5))
    plt.plot(num_features, scores, marker='o')
    
    # Setting the axis labels
    plt.xlabel('Number of Features')
    plt.ylabel('Score')
    
    # Set title
    plt.title('Number of Features vs. Score')
    
    # Show grid
    plt.grid(True)
    
    # Display the plot
    plt.show()

plot_features_vs_score(models)

## Questions:
Based on your work above, answer the following questions. 

1. When using AIC to score, what is the recommended model? What is the score this model receives? Does the graph confirm this?
2. When using BIC to score, what is the recommended model? What is the score this model receives? Does the graph confirm this?
3. When using $R^2$ to score, we know every additional variable will improve the model. What was the score received for the full model? Based on the graph, what model would you pick? What score did it receive?
4. Considering all of the results you've looked at, what model would you recommend using?

## Answers:

1. When using the AIC criterion, all 10 features are recommended to model the changes in the dependent variable. The score the model received was -162.40399146442417, which is significantly lower than the next best score. The graph seems to confirm the choice of using all 10 features since the 10th feature results in a dramatic drop in the score.

2. When using the BIC criterion, all 10 features are recommended to model the changes in the dependent variable. The score the model received was -133.74711941855514, which is significantly lower than the next best score. The graph seems to confirm the choice of using all 10 features since the 10th feature results in a dramatic drop in the score.

3. When using the coefficient of determination, the full model receives a value of $R^2 = 0.999999746381322$, which is similar to the next best model. When we look at the graph, we would likely deduce that 6 features are sufficient for modeling the changes in the dependent variable, namely $\beta_2, \beta_4, \beta_5, \beta_6, \beta_7$ and $\beta_{10}$.

4. Considering all the results above, it is likely best to consider the full model, especially because all coefficients are shown to be statistically significant. That said, special attention should be paid to features $X_2, X_4, X_5, X_6, X_7,$ and $X_{10}$, since they appear to model the majority of the change in $\hat y$. 